# Analysis Notebook - Hierarchical Bayesian Modelling

## **NOTE**:

We assume that you have cloned the analysis repository and have `cd` into the parent directory. Before starting with the analysis make sure you have first completed the dependencies set up by following the instructions described in the **`dependencies/README.md`** document. All paths defined in this Notebook are relative to the parent directory (repository). Please close this Notebook and start again by following the above guidelines if you have not completed the aforementioned steps.

## Prerequisite input files

Before starting the execution of the following code, make sure you have available in the folders `sbas/data` and `sbas/assets` the files listed below as prerequisites.

###  **`sbas/data`**.
The present analysis requires the following files to be present in the folder **`sbas/data`**.


- [x] The contents of `data.tar.gz` after unpacking them into the `sbas/data` folder with `tar xvzf data.tar.gz -C sbas/data `
- [x] `rmats_final.se.jc.ijc.txt`
- [x] `rmats_final.se.jc.sjc.txt`
- [x] `srr_pdata.csv` : the corrected GTEx data as created by the forked yarn and in the `annes-changes` branch https://github.com/TheJacksonLaboratory/yarn/tree/annes-changes with the SRR data as used in the `rMATS 3.2.5` experiment.


Additionally, the file `GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct` which is retrieved in the script from [`https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/`](https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz) and stored into the folder 
`sbas/data` as well.


### **`sbas/assets`**
The present analysis requires the following files to be present in the folder **`sbas/assets`**.

- [x] `tissues.tsv`: metadata file with information on which tissues will be used for analysis -- only using the `include` option for analysis
- [x] `splice-relevant-genes.txt`: list of RNA binding proteins that are annotated to splicing relevant functions from GO.

## Loading dependencies

If `conda` is available on your environment you can install the required dependencies by running the following commands:


```bash
time conda install -y r-base==3.6.2 &&
conda install -y r-ggplot2 r-ggsci r-coda r-rstan r-rjags r-compute.es r-snakecase &&
Rscript -e 'install.packages("runjags", repos = "https://cloud.r-project.org/")'
```



In [4]:
# Start the clock!
start_time <- Sys.time()

In [5]:
# dataviz dependencies
library(ggplot2)
library(ggsci)
library(grid)
library(gridExtra)
library(stringr)
library(snakecase)

# BDA2E-utilities dependencies
library(rstan)
library(parallel)
library(rjags)
library(runjags)
library(compute.es)

Previously used list of tissues to use for the Hierarchical Bayesian modelling:



```R
tissue.list<-c("Heart - Left Ventricle",
               "Breast - Mammary Tissue",
               "Brain - Cortex.Brain - Frontal Cortex (BA9).Brain - Anterior cingulate cortex (BA24)",
               "Adrenal Gland",
               "Adipose - Subcutaneous",
               "Muscle - Skeletal",
               "Thyroid",
               "Cells - Transformed fibroblasts",
               "Artery - Aorta",
               "Skin - Sun Exposed (Lower leg).Skin - Not Sun Exposed (Suprapubic)")
```

In [6]:
tissues_df <- readr::read_delim("../assets/tissues.tsv", delim = "\t")

Parsed with column specification:
cols(
  name = col_character(),
  female = col_double(),
  male = col_double(),
  include = col_double(),
  display.name = col_character()
)



In [7]:
tissue.list <- tissues_df$name[ tissues_df$include ==1]

In [8]:
message(length(tissue.list), " tissues")
cat(tissue.list, sep = "\n")

39 tissues



adipose_subcutaneous
adipose_visceral_omentum
adrenal_gland
artery_aorta
artery_coronary
artery_tibial
brain_caudate_basal_ganglia
brain_cerebellar_hemisphere
brain_cerebellum
brain_cortex
brain_frontal_cortex_ba_9
brain_hippocampus
brain_hypothalamus
brain_nucleus_accumbens_basal_ganglia
brain_putamen_basal_ganglia
brain_spinal_cord_cervical_c_1
breast_mammary_tissue
cells_cultured_fibroblasts
cells_ebv_transformed_lymphocytes
colon_sigmoid
colon_transverse
esophagus_gastroesophageal_junction
esophagus_mucosa
esophagus_muscularis
heart_atrial_appendage
heart_left_ventricle
liver
lung
muscle_skeletal
nerve_tibial
pancreas
pituitary
skin_not_sun_exposed_suprapubic
skin_sun_exposed_lower_leg
small_intestine_terminal_ileum
spleen
stomach
thyroid
whole_blood


In [9]:
tissue <- tissue.list[tissue_index]  #can be replaced with a loop or argument to choose a different tissue

In [10]:
tissue

[1] "adipose_subcutaneous"

## Pattern for choosing `topTable()` files from `limma`

```bash
# {as_site_type} + '_' + {tissue} + '_' + suffix_pattern 
se_skin_not_sun_exposed_suprapubic_AS_model_B_sex_as_events.csv
```

In [11]:
dataDir <- "../data/"
assetsDir <- "../assets/"
as_site_type <- "se"
suffix_pattern <- "AS_model_B_sex_as_events.csv"

file.with.de.results <- paste0(dataDir, as_site_type, "_", tissue, "_" , suffix_pattern  )
file.with.de.results
file.exists(file.with.de.results)
system( paste0("ls -l ", file.with.de.results), intern = TRUE )

[1] "../data/se_adipose_subcutaneous_AS_model_B_sex_as_events.csv"

[1] TRUE

[1] "-rw-r--r-- 1 jovyan users 4667129 Jun 18 16:57 ../data/se_adipose_subcutaneous_AS_model_B_sex_as_events.csv"

In [12]:
events.table         <- read.table(file.with.de.results, sep = ",")
head(events.table, 2)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
XIST-10149,-7.166835,1.0008990,-51.69164,1.877832e-222,7.253878e-218,436.5794
XIST-10154,-6.906147,0.7659794,-50.56963,7.990861e-218,1.543395e-213,428.6005


## Add annotation columns to the topTable dataframe:

The feature information is encoded in the topTable dataframe as rownames. The `ID` and `geneSymbol` variables have been combined in the following pattern:

```console
{geneSymbol}-{ID} 
```

- `ID`: everything **_after_** last occurence of hyphen `-`
example: 
```R
stringr::str_replace("apples - oranges - bananas", "^.+-", "")
```

```console
# output:

' bananas'
```

- `geneSymbol`: everything **_before_** last occurence of `-`
example: 

```R
sub('-[^-]*$', '',"apples - oranges - bananas")
```

```console
# output:

'apples - oranges '
```

```diff
- NOTE: The above solution covers the cases where a hyphen is part of the geneSymbol.
```

In [13]:
cols_initially <- colnames(events.table)
cols_initially

[1] "logFC"     "AveExpr"   "t"         "P.Value"   "adj.P.Val" "B"

In [14]:
events.table[["ID"]] <- stringr::str_replace(rownames(events.table),  "^.+-", "")
events.table[["gene_name"]] <- sub('-[^-]*$', '', rownames(events.table))

In [15]:
keepInOrderCols <- c("gene_name", "ID", cols_initially)

In [16]:
events.table <- events.table[ , keepInOrderCols ]

In [17]:
tail(events.table, 2)

,gene_name,ID,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MYL6-28855,MYL6,28855,0.01947220,10.22273,0.2629820,0.7926553,0.9651675,-6.553282
CD74-25493,CD74,25493,-0.01944741,10.19811,-0.1964588,0.8443180,0.9759174,-6.580949


## Define filepaths of required inputs

`file.with.de.results` has been defined above

In [18]:
rbp.table.name        <- paste0(assetsDir, "splice-relevant-genes.txt")
file.exists(rbp.table.name)

[1] TRUE

In [19]:
events.table.name     <- paste0(dataDir, "fromGTF.SE.txt")
file.exists(events.table.name)

[1] TRUE

In [20]:
inc.counts.file.name  <- paste0(dataDir, "rmats_final.se.jc.ijc.txt")
file.exists(inc.counts.file.name)

[1] TRUE

In [21]:
skip.counts.file.name <- paste0(dataDir, "rmats_final.se.jc.sjc.txt")
file.exists(skip.counts.file.name)

[1] TRUE

In [22]:
metadata.file.name    <- paste0(dataDir, "srr_pdata.csv")
file.exists(metadata.file.name)

[1] TRUE

In [23]:
expression.file.name  <- paste0(dataDir, "GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct")
file.exists(expression.file.name)

[1] TRUE

## Use the define filepaths to load/read in the tables 

Load the skip and inclusion count matrices, and the list of RNA binding proteins that are annotated to either:
- mRNA splicing, via spliceosome `(GO:0000398)`,
- regulation of mRNA splicing, via spliceosome `(GO:0048024)`, or 
- both. 

The table has the:
- Gene Symbol
- the Uniprot ID (`uprot.id`)
- the NCBI Gene ID (`gene.id`) and 
- boolean columns for being 
  - `S`=mRNA splicing, via spliceosome `(GO:0000398)` and 
  - `R`=regulation of mRNA splicing, via spliceosome `(GO:0048024)`.

### Filtering of the `topTable()` object

- `abs(events.table$logFC)>=log2(1.5)`
- `events.table$adj.P.Val<=0.05`

In [24]:
dim(events.table)
events.table <- events.table[abs(events.table$logFC)>=log2(1.5) & events.table$adj.P.Val<=0.05,]
dim(events.table)
head(events.table,2)

[1] 38629     8

[1] 61  8

,gene_name,ID,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
XIST-10149,XIST,10149,-7.166835,1.0008990,-51.69164,1.877832e-222,7.253878e-218,436.5794
XIST-10154,XIST,10154,-6.906147,0.7659794,-50.56963,7.990861e-218,1.543395e-213,428.6005


Make sure this command has been executed before `gunzip sbas/data/fromGTF.*` as the files are expected uncompressed.


In [25]:
annot.table  <- read.table(events.table.name,header=T)
dim(annot.table)
head(annot.table, 1)

[1] 42611    11

,ID,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,downstreamES,downstreamEE
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,ENSG00000034152.18,MAP2K3,chr17,+,21287990,21288091,21284709,21284969,21295674,21295769


In [26]:
merged.table <- merge(events.table, annot.table, by="ID")

In [27]:
dim(merged.table)
head(merged.table, 2)

[1] 61 18

,ID,gene_name,logFC,AveExpr,t,P.Value,adj.P.Val,B,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,downstreamES,downstreamEE
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>
1,10149,XIST,-7.166835,1.000899,-51.69164,1.877832e-222,7.253878e-218,436.5794,ENSG00000229807.11,XIST,chrX,-,73831065,73831274,73829067,73829231,73833237,73833374
2,10150,XIST,-3.020911,1.516307,-27.05427,8.819760e-106,8.517463e-102,216.1859,ENSG00000229807.11,XIST,chrX,-,73833237,73833374,73831065,73831274,73837439,73841474


In [28]:
rbp.table    <- read.table(rbp.table.name,sep="\t",header=TRUE)
dim(rbp.table)
head(rbp.table, 1)

[1] 351   6

,Gene,uprot.id,gene.id,S,R,omim
,<fct>,<fct>,<int>,<lgl>,<lgl>,<fct>
1,AAR2,Q9Y312,25980,TRUE,FALSE,n/a


Make sure this command has been executed before `gunzip sbas/data/rmats_final.se.jc.*jc.*` as the files are expected uncompressed.


In [29]:
inc.counts   <- as.data.frame(data.table::fread(inc.counts.file.name))
dim(inc.counts)
inc.counts[1:2,1:3]

[1] 42611  8674

,ID,SRR1068788,SRR1068808
,<int>,<int>,<int>
1,1,0,0
2,2,26,247


In [30]:
skip.counts  <- as.data.frame(data.table::fread(skip.counts.file.name))
dim(skip.counts)
skip.counts[1:2,1:3]

[1] 42611  8674

,ID,SRR1068788,SRR1068808
,<int>,<int>,<int>
1,1,2,0
2,2,0,0


## Check `dim()` of loaded objects

In [31]:
dim(events.table)
dim(annot.table)
dim(merged.table)
dim(rbp.table)
dim(inc.counts)
dim(skip.counts)

[1] 61  8

[1] 42611    11

[1] 61 18

[1] 351   6

[1] 42611  8674

[1] 42611  8674

## Read sample info

In [32]:
metadata.file.name
file.exists(metadata.file.name)
system(paste0("ls -l", " ../data/srr_pdata.csv"), intern = TRUE)

[1] "../data/srr_pdata.csv"

[1] TRUE

[1] "-rw-r--r-- 1 jovyan users 4230047 Jun 17 12:00 ../data/srr_pdata.csv"

In [33]:
meta.data <- readr::read_csv(metadata.file.name)
dim(meta.data)
head(meta.data, 1)

Parsed with column specification:
cols(
  .default = col_double(),
  SAMPID = col_character(),
  SMATSSCR = col_character(),
  SMCENTER = col_character(),
  SMPTHNTS = col_character(),
  SMTS = col_character(),
  SMTSD = col_character(),
  SMUBRID = col_character(),
  SMNABTCH = col_character(),
  SMNABTCHT = col_character(),
  SMNABTCHD = col_character(),
  SMGEBTCH = col_character(),
  SMGEBTCHD = col_character(),
  SMGEBTCHT = col_character(),
  SMAFRZE = col_character(),
  SMGTC = col_logical(),
  SMNUMGPS = col_logical(),
  SM550NRM = col_logical(),
  SM350NRM = col_logical(),
  SMMNCPB = col_logical(),
  SMMNCV = col_logical()
  # ... with 6 more columns
)

See spec(...) for full column specifications.



[1] 7872   68

SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS,SUBJID,SEX,AGE,DTHHRDY,SRR
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
GTEX.PW2O.0006.SM.2I3DV,NA,B1,NA,7.4,Blood,Whole Blood,13756,-126,NA,⋯,0.00351302,0.859573,NA,0,50.6829,GTEX-PW2O,1,20-29,0,SRR604002


In [34]:
meta.data$SMTSD[1:3]

[1] "Whole Blood"            "Lung"                   "Adipose - Subcutaneous"

In [35]:
meta.data[["SMTSD"]] <- as.character(meta.data[["SMTSD"]])

In [36]:
meta.data$SMTSD[1:3]

[1] "Whole Blood"            "Lung"                   "Adipose - Subcutaneous"

In [37]:
meta.data <- meta.data[ snakecase::to_snake_case(meta.data$SMTSD) == tissue,]

In [38]:
tissue
dim(meta.data)
head(meta.data,1)

[1] "adipose_subcutaneous"

[1] 294  68

SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS,SUBJID,SEX,AGE,DTHHRDY,SRR
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
GTEX.QESD.1526.SM.2S1QT,1,B1,2 pieces,7.8,Adipose Tissue,Adipose - Subcutaneous,2190,336,1304,⋯,0.00381426,0.652476,NA,0,50.3413,GTEX-QESD,1,20-29,0,SRR614191


In [39]:
# Undo snakecase of SMTSD
tissue
tissue <- unique(meta.data$SMTSD)
tissue

[1] "adipose_subcutaneous"

[1] "Adipose - Subcutaneous"

In [40]:
dim(inc.counts)
inc.counts   <- inc.counts[,colnames(inc.counts) %in% meta.data$SRR]
dim(inc.counts)

[1] 42611  8674

[1] 42611   294

In [41]:
dim(skip.counts)
skip.counts  <- skip.counts[,colnames(skip.counts) %in% meta.data$SRR]
dim(skip.counts)

[1] 42611  8674

[1] 42611   294

In [42]:
sd.threshold <- quantile(apply(inc.counts,1,sd)+apply(skip.counts,1,sd),0.95)
sd.threshold

95% 
115.1283

In [43]:
dim(skip.counts)
skip.counts  <- skip.counts[rownames(skip.counts) %in% merged.table$ID,]
dim(skip.counts)

[1] 42611   294

[1]  61 294

In [44]:
dim(inc.counts)
inc.counts   <- inc.counts[rownames(inc.counts) %in% merged.table$ID,]
dim(inc.counts)

[1] 42611   294

[1]  61 294

In [45]:
nrow(skip.counts)>100

[1] FALSE

In [46]:
if (nrow(skip.counts)>100)
{
  select.events <- apply(inc.counts,1,sd)+apply(skip.counts,1,sd)>sd.threshold
  inc.counts    <- inc.counts[select.events,]
  skip.counts   <- skip.counts[select.events,]
  merged.table  <- merged.table[select.events,]
}

In [47]:
dim(inc.counts)
dim(skip.counts)
dim(merged.table)

[1]  61 294

[1]  61 294

[1] 61 18

## Read expression data:

In [48]:
expression.file.name
file.exists(expression.file.name)

[1] "../data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct"

[1] TRUE

In [49]:
expression.mat <- read.table(expression.file.name, 
                             nrows = 1,
                             sep = "\t",
                             header = T,
                             skip = 2)

In [50]:
dim(expression.mat)
head(expression.mat, 2)

[1]     1 11690

,Name,Description,GTEX.1117F.0226.SM.5GZZ7,GTEX.111CU.1826.SM.5GZYN,GTEX.111FC.0226.SM.5N9B8,GTEX.111VG.2326.SM.5N9BK,GTEX.111YS.2426.SM.5GZZQ,GTEX.1122O.2026.SM.5NQ91,GTEX.1128S.2126.SM.5H12U,GTEX.113IC.0226.SM.5HL5C,⋯,GTEX.ZVE2.0006.SM.51MRW,GTEX.ZVP2.0005.SM.51MRK,GTEX.ZVT2.0005.SM.57WBW,GTEX.ZVT3.0006.SM.51MT9,GTEX.ZVT4.0006.SM.57WB8,GTEX.ZVTK.0006.SM.57WBK,GTEX.ZVZP.0006.SM.51MSW,GTEX.ZVZQ.0006.SM.51MR8,GTEX.ZXES.0005.SM.57WCB,GTEX.ZXG5.0005.SM.57WCN
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000223972.4,DDX11L1,0.1082,0.1158,0.02104,0.02329,0,0.04641,0.03076,0.09358,⋯,0.09012,0.1462,0.1045,0,0.6603,0.695,0.1213,0.4169,0.2355,0.145


In [51]:
colnames(expression.mat)[1:3]

[1] "Name"                     "Description"             
[3] "GTEX.1117F.0226.SM.5GZZ7"

In [52]:
colnames.expression.mat <- colnames(expression.mat)

In [53]:
length(colnames.expression.mat)
colnames.expression.mat[1:4]

[1] 11690

[1] "Name"                     "Description"             
[3] "GTEX.1117F.0226.SM.5GZZ7" "GTEX.111CU.1826.SM.5GZYN"

In [54]:
length(colnames.expression.mat)

[1] 11690

In [55]:
total.samples           <- length(colnames.expression.mat)
total.samples

[1] 11690

In [56]:
meta.data$SAMPID[1]
gsub("-","\\.",meta.data$SAMPID[1])

[1] "GTEX.QESD.1526.SM.2S1QT"

[1] "GTEX.QESD.1526.SM.2S1QT"

In [57]:
meta.data$SAMPID   <- gsub("-","\\.",meta.data$SAMPID)

In [58]:
dim(meta.data)
meta.data               <- meta.data[meta.data$SAMPID %in% colnames(expression.mat),]
dim(meta.data)

[1] 294  68

[1] 292  68

In [59]:
head(expression.mat)

,Name,Description,GTEX.1117F.0226.SM.5GZZ7,GTEX.111CU.1826.SM.5GZYN,GTEX.111FC.0226.SM.5N9B8,GTEX.111VG.2326.SM.5N9BK,GTEX.111YS.2426.SM.5GZZQ,GTEX.1122O.2026.SM.5NQ91,GTEX.1128S.2126.SM.5H12U,GTEX.113IC.0226.SM.5HL5C,⋯,GTEX.ZVE2.0006.SM.51MRW,GTEX.ZVP2.0005.SM.51MRK,GTEX.ZVT2.0005.SM.57WBW,GTEX.ZVT3.0006.SM.51MT9,GTEX.ZVT4.0006.SM.57WB8,GTEX.ZVTK.0006.SM.57WBK,GTEX.ZVZP.0006.SM.51MSW,GTEX.ZVZQ.0006.SM.51MR8,GTEX.ZXES.0005.SM.57WCB,GTEX.ZXG5.0005.SM.57WCN
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000223972.4,DDX11L1,0.1082,0.1158,0.02104,0.02329,0,0.04641,0.03076,0.09358,⋯,0.09012,0.1462,0.1045,0,0.6603,0.695,0.1213,0.4169,0.2355,0.145


In [60]:
meta.data <- meta.data[!duplicated(meta.data$SAMPID),]

In [61]:
dim(meta.data)

[1] 292  68

In [62]:
inc.counts <- inc.counts[,colnames(inc.counts) %in% meta.data$SRR]
dim(inc.counts)
head(inc.counts,1)

[1]  61 292

,SRR1069097,SRR1070184,SRR1070503,SRR1071453,SRR1072247,SRR1073581,SRR1075971,SRR1077038,SRR1078489,SRR1079522,⋯,SRR813680,SRR813824,SRR814387,SRR815869,SRR817329,SRR818797,SRR819459,SRR819820,SRR820755,SRR821715
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
343,65,161,96,129,57,285,30,42,11,7,⋯,136,40,54,232,9,169,165,834,164,266


In [63]:
skip.counts <- skip.counts[,colnames(skip.counts) %in% meta.data$SRR]
dim(skip.counts)
head(skip.counts, 1)

[1]  61 292

,SRR1069097,SRR1070184,SRR1070503,SRR1071453,SRR1072247,SRR1073581,SRR1075971,SRR1077038,SRR1078489,SRR1079522,⋯,SRR813680,SRR813824,SRR814387,SRR815869,SRR817329,SRR818797,SRR819459,SRR819820,SRR820755,SRR821715
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
343,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [64]:
meta.data <- meta.data[meta.data$SRR %in% colnames(inc.counts),]
dim(meta.data)
head(meta.data, 1)

[1] 292  68

SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS,SUBJID,SEX,AGE,DTHHRDY,SRR
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
GTEX.QESD.1526.SM.2S1QT,1,B1,2 pieces,7.8,Adipose Tissue,Adipose - Subcutaneous,2190,336,1304,⋯,0.00381426,0.652476,NA,0,50.3413,GTEX-QESD,1,20-29,0,SRR614191


In [65]:
colnames.expression.mat[1:4]

[1] "Name"                     "Description"             
[3] "GTEX.1117F.0226.SM.5GZZ7" "GTEX.111CU.1826.SM.5GZYN"

In [66]:
dim(expression.mat)
head(expression.mat, 1)

[1]     1 11690

,Name,Description,GTEX.1117F.0226.SM.5GZZ7,GTEX.111CU.1826.SM.5GZYN,GTEX.111FC.0226.SM.5N9B8,GTEX.111VG.2326.SM.5N9BK,GTEX.111YS.2426.SM.5GZZQ,GTEX.1122O.2026.SM.5NQ91,GTEX.1128S.2126.SM.5H12U,GTEX.113IC.0226.SM.5HL5C,⋯,GTEX.ZVE2.0006.SM.51MRW,GTEX.ZVP2.0005.SM.51MRK,GTEX.ZVT2.0005.SM.57WBW,GTEX.ZVT3.0006.SM.51MT9,GTEX.ZVT4.0006.SM.57WB8,GTEX.ZVTK.0006.SM.57WBK,GTEX.ZVZP.0006.SM.51MSW,GTEX.ZVZQ.0006.SM.51MR8,GTEX.ZXES.0005.SM.57WCB,GTEX.ZXG5.0005.SM.57WCN
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000223972.4,DDX11L1,0.1082,0.1158,0.02104,0.02329,0,0.04641,0.03076,0.09358,⋯,0.09012,0.1462,0.1045,0,0.6603,0.695,0.1213,0.4169,0.2355,0.145


In [67]:
tissue <- unique(meta.data$SMTSD [ meta.data$SMTSD == tissue])
tissue

[1] "Adipose - Subcutaneous"

In [68]:
col.in.tissue<-c()
for (col in colnames.expression.mat)
  
  col.in.tissue<-c(col.in.tissue, (col %in% meta.data$SAMPID) && (meta.data$SMTSD[which(meta.data$SAMPID==col)] %in% tissue) && (meta.data$SUBJID[which(meta.data$SAMPID==col)]!='GTEX-11ILO'))

In [69]:
length(col.in.tissue)
table(col.in.tissue)

[1] 11690

col.in.tissue
FALSE  TRUE 
11398   292 

In [70]:
length(colnames.expression.mat)
length(col.in.tissue)

col.in.tissue[1:3]

[1] 11690

[1] 11690

[1] FALSE FALSE  TRUE

In [71]:
# colClasses is used to skip columns
expression.mat <-read.table(expression.file.name, 
                            sep= "\t",
                            header = T,
                            skip = 2, 
                            colClasses = ifelse(col.in.tissue, "numeric", "NULL"))

In [72]:
length(col.in.tissue)

[1] 11690

## Read gene names:

In [73]:
dim(expression.mat)
expression.mat <- expression.mat[,order(match(colnames(expression.mat),meta.data$SAMPID))]
dim(expression.mat)

[1] 56202   292

[1] 56202   292

In [74]:
inc.counts     <- inc.counts[,order(match(colnames(inc.counts),meta.data$SRR))]
dim(inc.counts)

[1]  61 292

In [75]:
skip.counts    <- skip.counts[,order(match(colnames(skip.counts),meta.data$SRR))]
dim(skip.counts)

[1]  61 292

In [76]:
all.genes      <- read.table(expression.file.name,sep="\t",header=T,skip=2,colClasses = c(rep("character", 2), rep("NULL", total.samples-2)))
dim(all.genes)
head(all.genes, 2)

[1] 56202     2

,Name,Description
,<chr>,<chr>
1,ENSG00000223972.4,DDX11L1
2,ENSG00000227232.4,WASH7P


In [77]:
expression.mat <- expression.mat[!duplicated(all.genes$Description),]
dim(expression.mat)
head(expression.mat,2)

[1] 54271   292

,GTEX.QESD.1526.SM.2S1QT,GTEX.QEG5.0326.SM.2S1PB,GTEX.SNMC.1326.SM.2XCFK,GTEX.O5YT.0226.SM.32PK5,GTEX.ZZ64.1626.SM.5E43W,GTEX.YJ8A.0526.SM.5IFHT,GTEX.144GM.1926.SM.5LUAN,GTEX.ZTX8.1526.SM.5N9GI,GTEX.T5JW.1726.SM.3GADN,GTEX.PWCY.1926.SM.3NB25,⋯,GTEX.11TTK.0226.SM.5N9EC,GTEX.13X6K.0226.SM.5QGPB,GTEX.SSA3.0226.SM.32QPN,GTEX.TMZS.0226.SM.3DB9N,GTEX.Y8E5.0226.SM.57WCM,GTEX.113IC.0226.SM.5HL5C,GTEX.13N1W.0226.SM.5K7W6,GTEX.13SLW.0226.SM.5S2NA,GTEX.13IVO.0226.SM.5LZXU,GTEX.X638.0226.SM.47JZ9
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.04424,0.07382,0.1254,0.00,0.1541,0.09404,0.068,0.01964,0.1687,0.06077,⋯,0.07889,0.0,0.06643,0.06056,0.1136,0.09358,0.04205,0.000,0.1006,0.09333
2,6.90800,11.21000,10.2500,13.36,14.3300,7.02500,12.350,9.59700,7.5320,9.15700,⋯,14.58000,10.3,15.32000,5.95400,13.8200,13.56000,15.74000,7.364,13.9700,24.24000


In [78]:
all.genes      <- all.genes[!duplicated(all.genes$Description),]
dim(all.genes)

[1] 54271     2

In [79]:
skip.counts    <- skip.counts[merged.table$geneSymbol %in% all.genes$Description,]
dim(skip.counts)

[1]  58 292

In [80]:
inc.counts     <- inc.counts[merged.table$geneSymbol %in% all.genes$Description,]
dim(inc.counts)

[1]  58 292

In [81]:
merged.table   <- merged.table[merged.table$geneSymbol %in% all.genes$Description,]
dim(merged.table)

[1] 58 18

In [82]:
gene.names     <- unique(merged.table$geneSymbol)
length(gene.names)

[1] 44

In [83]:
expression.mat <- expression.mat[all.genes$Description %in% c(as.character(rbp.table$Gene),as.character(gene.names)),]
dim(expression.mat)

[1] 383 292

In [84]:
rownames.expression.mat <-all.genes$Description[all.genes$Description %in% c(as.character(rbp.table$Gene),as.character(gene.names))]
length(rownames.expression.mat)

[1] 383

In [85]:
expression.mat <-expression.mat[!duplicated(rownames.expression.mat),]
dim(expression.mat)

[1] 383 292

In [86]:
rownames.expression.mat <-rownames.expression.mat[!duplicated(rownames.expression.mat)]
length(rownames.expression.mat)

[1] 383

## Prepare expression of genes and RBPS:

In [87]:
num.events     <- nrow(merged.table)
num.events

[1] 58

In [88]:
event.to.gene  <- c()

In [89]:
gexp           <- expression.mat[rownames.expression.mat %in% gene.names,]
dim(gexp)

[1]  44 292

In [90]:
rownames(gexp) <- rownames.expression.mat[rownames.expression.mat %in% gene.names]

In [91]:
gexp           <- gexp[order(match(rownames(gexp),gene.names)),]
dim(gexp)
head(gexp,2)

[1]  44 292

,GTEX.QESD.1526.SM.2S1QT,GTEX.QEG5.0326.SM.2S1PB,GTEX.SNMC.1326.SM.2XCFK,GTEX.O5YT.0226.SM.32PK5,GTEX.ZZ64.1626.SM.5E43W,GTEX.YJ8A.0526.SM.5IFHT,GTEX.144GM.1926.SM.5LUAN,GTEX.ZTX8.1526.SM.5N9GI,GTEX.T5JW.1726.SM.3GADN,GTEX.PWCY.1926.SM.3NB25,⋯,GTEX.11TTK.0226.SM.5N9EC,GTEX.13X6K.0226.SM.5QGPB,GTEX.SSA3.0226.SM.32QPN,GTEX.TMZS.0226.SM.3DB9N,GTEX.Y8E5.0226.SM.57WCM,GTEX.113IC.0226.SM.5HL5C,GTEX.13N1W.0226.SM.5K7W6,GTEX.13SLW.0226.SM.5S2NA,GTEX.13IVO.0226.SM.5LZXU,GTEX.X638.0226.SM.47JZ9
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
XIST,0.0425,0.1722,0.004303,0.3899,0.1726,0.04516,0.05598,0.03773,18.260,38.170,⋯,50.9900,51.120,0.1155,0.07202,0.02859,0.06848,0.06635,0.0337,0.007891,39.530
BDH1,1.7740,3.7470,4.705000,5.2410,5.6460,5.08700,3.96000,3.98100,6.544,9.398,⋯,0.7406,3.635,3.2980,2.68500,7.92000,2.77900,2.66300,3.5430,4.854000,1.975


In [92]:
gexp           <- log2(gexp+0.5)

In [93]:
gexp           <- gexp-rowMeans(gexp)

In [94]:
gexp[apply(gexp,1,sd)>0,] <- gexp[apply(gexp,1,sd)>0,]/apply(gexp[apply(gexp,1,sd)>0,],1,sd)

In [95]:
rexp           <- expression.mat[rownames.expression.mat %in% rbp.table$Gene,]

In [96]:
rownames(rexp) <- rownames.expression.mat[rownames.expression.mat %in% rbp.table$Gene]

In [97]:
rexp           <- rexp[order(match(rownames(rexp),rbp.table$Gene)),]

In [98]:
rexp           <- log2(rexp+0.5)

In [99]:
rexp           <- rexp-rowMeans(rexp)

In [100]:
rexp           <- rexp/apply(rexp,1,function(v){ifelse(sum(v==v[1])<length(v),sd(v),1)})

In [101]:
for (i in (1:num.events))
  event.to.gene<-c(event.to.gene,which(unique(merged.table$geneSymbol)==merged.table[i,"geneSymbol"]))
    sex<-ifelse(meta.data$SEX==1,1,0)

In [102]:
sex[1:4]
table(sex)

[1] 1 1 1 1

sex
  0   1 
104 188 

In [103]:
end_time <- Sys.time()
end_time - start_time

Time difference of 3.06765 mins

## Run stan:

In [104]:
dataList = list(
  as = round(skip.counts) ,   #skip event counts across experiments
  c = round(skip.counts+inc.counts)    , #total counts for event, i.e. skip+inclusion, across experiments
  gexp = gexp, #read counts for genes (from gtex, take the raw counts) across experiments
  rexp = rexp, #read counts for RBPs (from gtex, take the raw counts)
  event_to_gene = event.to.gene,  #the gene index for each event (1 to the number of distinct genes) 
  Nrbp = nrow(rexp), #number of RBPs
  Nevents = nrow(merged.table),  #most varying AS events in 
  Nexp = ncol(expression.mat),#number of experiments such that we measured each event, gene and RBP in each experiment
  Ngenes = nrow(gexp),
  sex=sex
)


modelString = "
data {
int<lower=0> Nevents;
int<lower=0> Nexp;
int<lower=0> Nrbp;
int<lower=0> Ngenes;
int<lower=0> as[Nevents,Nexp] ;
int<lower=0> c[Nevents,Nexp] ;
matrix[Ngenes,Nexp] gexp ; 
matrix[Nrbp,Nexp] rexp ; 
int<lower=0> event_to_gene[Nevents];
int<lower=0,upper=1> sex[Nexp];

}


parameters {
real beta0[Nevents] ;
real beta1[Nevents] ;
matrix[Nevents,Nrbp] beta2 ;
real beta3[Nevents];
real beta4[Nrbp];

}
model {

for ( i in 1:Nexp ) {  


    for ( j in 1:Nevents ) if (c[j,i]>0) { 

      as[j,i] ~ binomial(c[j,i], inv_logit(beta0[j]+beta1[j]*sex[i]+dot_product(beta2[j,],rexp[,i])+beta3[j]*gexp[event_to_gene[j],i] ) );

  }
}

for (k in 1:Nrbp){

  for ( j in 1:Nevents ) { 

        beta2[j,k] ~normal(beta4[k],1);
  }

  beta4[k]~normal(0,1);

}


for ( j in 1:Nevents ) { 

    beta1[j] ~ normal(0,1);
    beta0[j] ~ normal(0,1);
    beta3[j] ~ normal(0,1);
  }

}
"

# Start the clock!
start_time <- Sys.time()

stanDso <- rstan::stan_model( model_code=modelString ) 
stanFit <- sampling( object=stanDso , 
                    data = dataList , 
                    chains = 2 , #3
                    iter = 8, #8000
                    warmup = 6, #6000
                    thin = 1,
                    init = 0, 
                    cores = parallel::detectCores() - 2 )

mcmcCoda = coda::mcmc.list( lapply( 1:ncol(stanFit) , function(x) { mcmc(as.array(stanFit)[,x,]) } ) )

end_time <- Sys.time()
end_time - start_time

Warning message:
“There were 4 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is NA, indicating chains have not mixed.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#tail-ess”


Time difference of 1.792077 mins

## Save R objects

In [105]:
save.image(file = "notebook.RData")
file.exists("notebook.RData")
system("pwd && ls -l notebook.RData", intern = TRUE)

[1] TRUE

[1] "/mnt/shared/gcp-user/session_data/sbas/jupyter"               
[2] "-rw-r--r-- 1 jovyan users 8084043 Jun 26 12:34 notebook.RData"

## Metadata

For replicability and reproducibility purposes, we also print the following metadata:

1. Checksums of **"artefacts"**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

### 1. Checksums with the sha256 algorithm

In [106]:
figure_id       <- "bayesian-modeling"

message("Generating sha256 checksums of the artefacts in the `..data/` directory .. ")
system(paste0("cd ../data/ && find . -type f -exec sha256sum {} \\; > ../metadata/",  figure_id, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

data.table::fread(paste0("../metadata/", figure_id, "_sha256sums.txt"), header = FALSE, col.names = c("sha256sum", "file"))

Generating sha256 checksums of the artefacts in the `..data/` directory .. 



character(0)

Done!




sha256sum,file
<chr>,<chr>
6f2b3dc37ab2186f7cb6d317d80d9f70e9cd1c2bcab3cfc86c7de5c5e76e2514,./mxe_spleen_AS_model_B_sex.csv
3d378262fa13e905cfa51d6ec2a88f01965d82dd54357131771c0a95c226805a,./mxe_brain_cerebellum_AS_model_A_ijc_wo_DGE_sex_gene_set.txt
94a7fa75a9c0072a745bb513040e778f90961310fc3751b90141b62a06429903,./liver_DGE.csv
c04f787ffba8d422a66d2e807ece6ac8c6900287572f31105a5549c4021faf81,./ri_pancreas_AS_model_A_ijc_wo_DGE_sex.csv
b4776f57f9780ef4c2068f55a9a56e02f2544d70438ce16de8a98a454bbaa14e,./ri_breast_mammary_tissue_AS_model_A_ijc_wo_DGE_sex_refined.csv
f6c59cceb70e2f036dc4ad705125a10412d22dca8dc2b6b2f8ebadd4aaf98280,./mxe_spleen_AS_model_A_sjc_sex.csv
2b5e26957f499a525c03053e7b5362681e1ebc7e71f0ae69e621153e56cbc3b9,./se_lung_AS_model_A_ijc_wo_DGE_sex.csv
cf6e8ed384c002dd3beaf00318db269252e11e447f57a2829c87c597dacc6dfb,./se_whole_blood_AS_model_B_sex_as_events_universe.txt
63c22570491153b15c881dd82928c26491fac0bd246beb1de65cbf81d9049911,./a5ss_thyroid_AS_model_B_sex_as_events_universe.txt


### 2. Libraries metadata

In [107]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", figure_id, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", figure_id ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]

Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..

Done!


Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..

Done!




 setting  value                       
 version  R version 3.6.2 (2019-12-12)
 os       Ubuntu 18.04.3 LTS          
 system   x86_64, linux-gnu           
 ui       X11                         
 language en_US.UTF-8                 
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       Etc/UTC                     
 date     2020-06-26                  

,package,ondiskversion,loadedversion,path,loadedpath,attached,is_base,date,source,md5ok,library
,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<lgl>,<fct>
coda,coda,0.19.3,0.19-3,/opt/conda/lib/R/library/coda,/opt/conda/lib/R/library/coda,TRUE,FALSE,2019-07-05,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
compute.es,compute.es,0.2.5,0.2-5,/opt/conda/lib/R/library/compute.es,/opt/conda/lib/R/library/compute.es,TRUE,FALSE,2020-04-01,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
ggplot2,ggplot2,3.3.2,3.3.2,/opt/conda/lib/R/library/ggplot2,/opt/conda/lib/R/library/ggplot2,TRUE,FALSE,2020-06-19,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
ggsci,ggsci,2.9,2.9,/opt/conda/lib/R/library/ggsci,/opt/conda/lib/R/library/ggsci,TRUE,FALSE,2018-05-14,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
gridExtra,gridExtra,2.3,2.3,/opt/conda/lib/R/library/gridExtra,/opt/conda/lib/R/library/gridExtra,TRUE,FALSE,2017-09-09,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
rjags,rjags,4.10,4-10,/opt/conda/lib/R/library/rjags,/opt/conda/lib/R/library/rjags,TRUE,FALSE,2019-11-06,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
rstan,rstan,2.19.3,2.19.3,/opt/conda/lib/R/library/rstan,/opt/conda/lib/R/library/rstan,TRUE,FALSE,2020-02-11,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
runjags,runjags,2.0.4.6,2.0.4-6,/opt/conda/lib/R/library/runjags,/opt/conda/lib/R/library/runjags,TRUE,FALSE,2019-12-17,CRAN (R 3.6.2),NA,/opt/conda/lib/R/library
snakecase,snakecase,0.11.0,0.11.0,/opt/conda/lib/R/library/snakecase,/opt/conda/lib/R/library/snakecase,TRUE,FALSE,2019-05-25,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
